<a href="https://colab.research.google.com/github/JuarezEmmanuel/Data-Science-Python/blob/main/Ventas_Llantas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción
Se tiene un set de datos de una empresa líder en Colombia en la venta de llantas, este contiene las ventas desde 2016 al 2020, mediante el uso de Python se pretende predecir la gestión de inventarios tomando en cuenta solo lubricantes . El cliente solicita que se realice un análisis que permita cubrir la demanda de los clientes para el año 2021, se debe tomar en cuenta que existe en cada tienda un espacio de bodega, en el cual no se debe tener stock excesivo, por lo que es importante predecir la cantidad máxima y mínima.

Juarez Emmanuel

## Carga de Librerias

In [1]:
!pip install -U gdown  #  descargar el archivo directamente desde Google Drive

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

## Lectura de Archivo
- Ubicacion del archivo
    . https://docs.google.com/spreadsheets/d/1kFa35DYNnfLzAAMEwE7pGwa0RiZN48_n/edit?usp=drive_link&ouid=110546804701229744351&rtpof=true&sd=true
- Venta_Llantas

In [2]:
!gdown 1kFa35DYNnfLzAAMEwE7pGwa0RiZN48_n

Downloading...
From: https://drive.google.com/uc?id=1kFa35DYNnfLzAAMEwE7pGwa0RiZN48_n
To: /content/venta_llantas.xlsx
100% 10.7M/10.7M [00:00<00:00, 11.1MB/s]


In [ ]:
df = pd.read_excel('venta_llantas.xlsx')

In [ ]:
df.info()

In [ ]:
df.head (10)

In [ ]:
df = df[df['Familia'] == 'LUBRICANTES']
df.head()

In [ ]:
df.info()

In [ ]:
# Valores Nulos por Columna
df.isnull().sum()

In [ ]:
# solo meses completos
df = df[df['Fecha'] < '2020-03-01']

In [ ]:
# Convertimos la columna 'Fecha' en formato datetime
df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce')

# Extraer el año de la columna 'Fecha' para agrupar las ventas anuales
df['Year'] = df['Fecha'].dt.year

# Agrupar por año y descripción de la llanta, sumando las cantidades vendidas por año. Luego pivotea la tabla reemplazando faltantes por 0
sales_by_tire_size = df.groupby(['Year', 'Descripcion'])['Cantidad'].sum().unstack(fill_value=0)

# Filtrar las ventas históricas para los años 2016-2020
historical_sales = sales_by_tire_size.loc[2016:2020]

# Extrapolación de 2016 y 2020
# 2016 solo tiene datos desde septiembre, por lo tanto multiplicamos por 3 (para extrapolar a todo el año)
historical_sales.loc[2016] = historical_sales.loc[2016] * 3

# 2020 solo tiene datos hasta febrero, por lo tanto multiplicamos por 6 (para extrapolar a todo el año)
historical_sales.loc[2020] = historical_sales.loc[2020] * 6

# Cálculo de estadísticas para los años completos 2016-2020
summary_stats_adjusted = historical_sales.loc[2016:2020].describe().T[['mean', 'min', 'max', 'std']]

In [ ]:
# Predicción para 2021 con un aumento conservador del 10% debido a la recuperación post-COVID
summary_stats_adjusted['predicted_2021'] = summary_stats_adjusted['mean'] * 1.1

In [ ]:
# Cálculo del intervalo de confianza al 95%
confidence_level = 0.95
z_score = stats.norm.ppf((1 + confidence_level) / 2)

# Intervalo de confianza: mean ± z * (std / sqrt(n))
# Donde n es el número de años (5 años: 2016, 2017, 2018 y 2019 , 2020)
n = 5
summary_stats_adjusted['ci_lower'] = summary_stats_adjusted['mean'] - z_score * (summary_stats_adjusted['std'] / np.sqrt(n))
summary_stats_adjusted['ci_upper'] = summary_stats_adjusted['mean'] + z_score * (summary_stats_adjusted['std'] / np.sqrt(n))

# Agregar las ventas históricas de cada año (2016-2020) al archivo final
summary_stats_adjusted = summary_stats_adjusted.join(historical_sales.loc[2016:2020].T)

In [ ]:
summary_stats_adjusted

In [ ]:
summary_stats_adjusted['P_2021'] = summary_stats_adjusted['predicted_2021'].round()
summary_stats_adjusted['P_min'] = summary_stats_adjusted['ci_lower'].apply(lambda x: 0 if x < 0 else round(x))
summary_stats_adjusted['P_max'] = summary_stats_adjusted['ci_upper'].apply(lambda x: 0 if x < 0 else round(x))

In [ ]:
# Guardar los resultados en un archivo CSV con predicciones, intervalos de confianza y ventas históricas
summary_stats_adjusted.to_csv('prediccion_ajustada_inventario_2021_con_intervalos.csv', index=True)

In [ ]:
# Guardar los resultados en un archivo CSV con predicciones, intervalos de confianza y ventas históricas
summary_stats_adjusted.to_csv('prediccion_ajustada_inventario_2021_con_intervalos.tsv', sep = '|')

In [ ]:
# Mostrar las predicciones ajustadas de inventario para 2021
print("\nPredicciones Ajustadas de Inventario 2021 con Intervalos de Confianza:")
summary_stats_adjusted[[2017, 2018, 2019, 2020, 'P_2021', 'P_min', 'P_max']]

# Resultados:

* Descripción : Nombre del producto o tipo de llanta (ejemplo: "10W30 GALON HC SPECIAL TEC AA 4L LIQUI MOLY").

* Mean (media) : Promedio de las ventas anuales del producto entre 2016 y 2019.

* Min (mínimo) : La cantidad mínima de ventas registradas en un año durante el periodo 2016-2019. En el caso del producto "10W30 GALON HC SPECIAL TEC AA 4L LIQUI MOLY", en al menos un año no se vendió nada (0 unidades).

* Max (máximo) : La cantidad máxima de ventas registradas en un año.

* Std (desviación estándar) : Medida de cantidad variación o dispersión hay en las ventas anuales. Cuanto mayor sea este valor, más dispersas están las ventas a lo largo de los años.

* Predicted_2021 (Predicción 2021) : predicción de la cantidad de unidades que se venderán en 2021, calculada como el promedio de ventas entre 2016 y 2019 con un aumento del 10% para ajustar posibles crecimientos post-COVID.

* CI_Lower (Límite inferior del intervalo de confianza) : El límite inferior del intervalo de confianza del 95% para las ventas promedio. Si el límite inferior es negativo, significa que hay una gran incertidumbre en las predicciones debido a la alta variabilidad de las ventas históricas.

* CI_Upper (Límite superior del intervalo de confianza) : El límite superior del intervalo de confianza del 95% para las ventas promedio.

* 2016, 2017, 2018, 2019, 2020 : Estas columnas muestran las cantidades vendidas de cada producto en esos años.

El análisis muestra productos con alta variabilidad en ventas, lo que sugiere cautela al planificar el inventario, especialmente para productos con grandes fluctuaciones históricas.